# 🚀 股票预测模型完整训练流程

**流程概览**：
1. 📥 加载智能体生成的数据
2. 🔄 数据预处理和特征工程
3. 🧠 Stage 1: 双输出SST模型训练（T日 + T+1日预测）
4. 🔍 Stage 2: 特征提取（Attention + Encoder + 残差）
5. ⏰ Stage 3: 时序模型训练（iTransformer + LSTM + GRU）
6. 📊 Stage 4: 模型效果验证和对比

**目标**：预测指定股票的T+1日涨跌幅

---

**状态**: 🚧 开发测试中

## 📦 环境准备

In [ ]:
# 克隆项目（如果还没有）
!git clone https://github.com/FTF1990/Quant-Stock-Transformer.git /content/Quant-Stock-Transformer 2>/dev/null || echo "项目已存在"
%cd /content/Quant-Stock-Transformer

# 安装依赖
!pip install -q torch pandas numpy scikit-learn matplotlib seaborn tqdm

print("✓ 环境准备完成")

In [ ]:
import sys
import json
import pickle
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 添加项目路径
sys.path.append('/content/Quant-Stock-Transformer')

# 导入项目模块
from models.spatial_feature_extractor import SpatialFeatureExtractor

# 设置随机种子
torch.manual_seed(42)
np.random.seed(42)

# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✓ 使用设备: {device}")

# 设置绘图样式
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

## 📥 Step 1: 加载数据

加载智能体生成的股票列表和历史数据

In [ ]:
# ============================================================================
# 配置：指定数据路径和目标股票
# ============================================================================

STOCKS_JSON_PATH = "selected_stocks.json"  # 智能体生成的股票列表
HISTORICAL_DATA_PATH = "historical_data.pkl"  # 抓取的历史数据

TARGET_MARKET = "CN"  # 目标市场
TARGET_STOCK_SYMBOL = "600519"  # 要预测的股票（贵州茅台示例）

# ============================================================================
# 加载数据
# ============================================================================

# 加载股票列表
with open(STOCKS_JSON_PATH, 'r', encoding='utf-8') as f:
    selected_stocks = json.load(f)

print(f"✓ 加载股票列表: {sum(len(v) for v in selected_stocks.values())}只股票")
for market, stocks in selected_stocks.items():
    print(f"  {market}: {len(stocks)}只")

# 加载历史数据
with open(HISTORICAL_DATA_PATH, 'rb') as f:
    historical_data = pickle.load(f)

print(f"\n✓ 加载历史数据完成")

# 检查目标股票
if TARGET_MARKET not in historical_data:
    raise ValueError(f"市场{TARGET_MARKET}数据不存在")

if TARGET_STOCK_SYMBOL not in historical_data[TARGET_MARKET]:
    raise ValueError(f"股票{TARGET_STOCK_SYMBOL}数据不存在")

print(f"\n✓ 目标股票: {TARGET_MARKET} - {TARGET_STOCK_SYMBOL}")
print(f"  数据条数: {len(historical_data[TARGET_MARKET][TARGET_STOCK_SYMBOL])}")

## 🔄 Step 2: 数据预处理

将原始数据转换为模型可用的格式

In [ ]:
class StockDataProcessor:
    """股票数据预处理器"""
    
    def __init__(self, historical_data: Dict, target_market: str, target_stock: str):
        self.historical_data = historical_data
        self.target_market = target_market
        self.target_stock = target_stock
        self.scaler = StandardScaler()
        
    def prepare_training_data(self, sequence_length: int = 60):
        """
        准备训练数据
        
        Returns:
            boundary_conditions: [N, num_boundary_sensors] - 边界条件（指数+股票）
            targets_T: [N, 1] - T日目标（当日收益率）
            targets_T1: [N, 1] - T+1日目标（次日收益率）
            dates: [N] - 日期索引
        """
        print("🔄 开始数据预处理...")
        
        # 获取市场数据
        market_data = self.historical_data[self.target_market]
        
        # 获取大盘指数
        index_df = market_data.get('_INDEX_')
        if index_df is None:
            print("  警告: 无大盘指数数据")
        
        # 获取目标股票数据
        target_df = market_data[self.target_stock].copy()
        
        # 计算收益率
        target_df['return'] = target_df['Close'].pct_change()
        target_df['return_next'] = target_df['return'].shift(-1)  # T+1日收益率
        
        # 移除NaN
        target_df = target_df.dropna()
        
        print(f"  ✓ 目标股票数据: {len(target_df)}条")
        
        # 构建边界条件（简化版：使用目标股票的历史数据作为特征）
        # 在实际应用中，这里应该包含大盘、板块、龙头股等
        boundary_features = []
        targets_T = []
        targets_T1 = []
        dates = []
        
        # 使用滑动窗口
        for i in range(len(target_df) - 1):
            # 边界条件：当日的OHLCV
            boundary = [
                target_df['Open'].iloc[i],
                target_df['High'].iloc[i],
                target_df['Low'].iloc[i],
                target_df['Close'].iloc[i],
                target_df['Volume'].iloc[i]
            ]
            
            # 添加大盘指数（如果有）
            if index_df is not None and len(index_df) > i:
                boundary.extend([
                    index_df['Close'].iloc[i]
                ])
            
            boundary_features.append(boundary)
            targets_T.append(target_df['return'].iloc[i])
            targets_T1.append(target_df['return_next'].iloc[i])
            dates.append(target_df.index[i])
        
        # 转换为numpy数组
        boundary_features = np.array(boundary_features, dtype=np.float32)
        targets_T = np.array(targets_T, dtype=np.float32).reshape(-1, 1)
        targets_T1 = np.array(targets_T1, dtype=np.float32).reshape(-1, 1)
        
        # 标准化边界条件
        boundary_features = self.scaler.fit_transform(boundary_features)
        
        print(f"  ✓ 边界条件形状: {boundary_features.shape}")
        print(f"  ✓ T日目标形状: {targets_T.shape}")
        print(f"  ✓ T+1日目标形状: {targets_T1.shape}")
        
        return boundary_features, targets_T, targets_T1, dates


# 实例化处理器
processor = StockDataProcessor(
    historical_data=historical_data,
    target_market=TARGET_MARKET,
    target_stock=TARGET_STOCK_SYMBOL
)

# 准备数据
X, y_T, y_T1, dates = processor.prepare_training_data()

print(f"\n✓ 数据预处理完成")
print(f"  总样本数: {len(X)}")
print(f"  特征维度: {X.shape[1]}")

## 📊 Step 3: 数据集划分

In [ ]:
# 时间序列数据：按时间顺序划分
train_size = int(0.7 * len(X))
val_size = int(0.15 * len(X))

X_train = X[:train_size]
y_T_train = y_T[:train_size]
y_T1_train = y_T1[:train_size]

X_val = X[train_size:train_size+val_size]
y_T_val = y_T[train_size:train_size+val_size]
y_T1_val = y_T1[train_size:train_size+val_size]

X_test = X[train_size+val_size:]
y_T_test = y_T[train_size+val_size:]
y_T1_test = y_T1[train_size+val_size:]

print(f"训练集: {len(X_train)} 样本")
print(f"验证集: {len(X_val)} 样本")
print(f"测试集: {len(X_test)} 样本")

## 🧠 Stage 1: 双输出SST模型训练

训练同时预测T日和T+1日收益率的SST模型

In [ ]:
# 从示例代码导入双输出SST
# （这里简化实现，完整版参考 examples/extract_sst_internals_demo.py）

class DualOutputSST(SpatialFeatureExtractor):
    """双输出SST"""
    
    def __init__(self, num_boundary_sensors, num_target_sensors, **kwargs):
        super().__init__(num_boundary_sensors, num_target_sensors, **kwargs)
        
        # 双输出头
        self.output_projection_T = nn.Linear(self.d_model, num_target_sensors)
        self.output_projection_T1 = nn.Linear(self.d_model, num_target_sensors)
        
        nn.init.xavier_uniform_(self.output_projection_T.weight)
        nn.init.xavier_uniform_(self.output_projection_T1.weight)
    
    def forward(self, boundary_conditions):
        batch_size = boundary_conditions.shape[0]
        
        x = boundary_conditions.unsqueeze(-1)
        x = self.boundary_embedding(x) + self.boundary_position_encoding.unsqueeze(0)
        x = self.transformer(x)
        x_pooled = x.permute(0, 2, 1)
        x_pooled = self.global_pool(x_pooled).squeeze(-1)
        
        pred_T = self.output_projection_T(x_pooled)
        pred_T1 = self.output_projection_T1(x_pooled)
        
        return pred_T, pred_T1
    
    def forward_with_features(self, boundary_conditions, **kwargs):
        batch_size = boundary_conditions.shape[0]
        features = {}
        
        x = boundary_conditions.unsqueeze(-1)
        x = self.boundary_embedding(x) + self.boundary_position_encoding.unsqueeze(0)
        features['embeddings'] = x.clone()
        
        if kwargs.get('return_attention', True):
            encoder_output, attention_weights = self._forward_transformer_with_attention(x)
            features['attention_weights'] = attention_weights
        else:
            encoder_output = self.transformer(x)
        
        features['encoder_output'] = encoder_output
        
        x_pooled = encoder_output.permute(0, 2, 1)
        x_pooled = self.global_pool(x_pooled).squeeze(-1)
        features['pooled_features'] = x_pooled
        
        pred_T = self.output_projection_T(x_pooled)
        pred_T1 = self.output_projection_T1(x_pooled)
        
        return (pred_T, pred_T1), features


# 创建模型
num_features = X.shape[1]
sst_model = DualOutputSST(
    num_boundary_sensors=num_features,
    num_target_sensors=1,
    d_model=128,
    nhead=8,
    num_layers=3,
    dropout=0.1,
    enable_feature_extraction=True
).to(device)

print(f"✓ SST模型创建完成")
print(f"  参数量: {sum(p.numel() for p in sst_model.parameters()):,}")

In [ ]:
# 训练SST模型
def train_sst(
    model, 
    X_train, y_T_train, y_T1_train,
    X_val, y_T_val, y_T1_val,
    epochs=50,
    batch_size=32,
    lr=0.001
):
    """训练双输出SST模型"""
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    
    # 转换为tensor
    X_train_t = torch.FloatTensor(X_train).to(device)
    y_T_train_t = torch.FloatTensor(y_T_train).to(device)
    y_T1_train_t = torch.FloatTensor(y_T1_train).to(device)
    
    X_val_t = torch.FloatTensor(X_val).to(device)
    y_T_val_t = torch.FloatTensor(y_T_val).to(device)
    y_T1_val_t = torch.FloatTensor(y_T1_val).to(device)
    
    history = {'train_loss': [], 'val_loss': []}
    
    print("\n🚀 开始训练SST模型...\n")
    
    for epoch in range(epochs):
        model.train()
        
        # 训练
        epoch_loss = 0
        num_batches = (len(X_train) + batch_size - 1) // batch_size
        
        for i in range(num_batches):
            start_idx = i * batch_size
            end_idx = min((i + 1) * batch_size, len(X_train))
            
            batch_X = X_train_t[start_idx:end_idx]
            batch_y_T = y_T_train_t[start_idx:end_idx]
            batch_y_T1 = y_T1_train_t[start_idx:end_idx]
            
            optimizer.zero_grad()
            
            pred_T, pred_T1 = model(batch_X)
            
            loss_T = criterion(pred_T, batch_y_T)
            loss_T1 = criterion(pred_T1, batch_y_T1)
            loss = loss_T + loss_T1
            
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        
        epoch_loss /= num_batches
        
        # 验证
        model.eval()
        with torch.no_grad():
            val_pred_T, val_pred_T1 = model(X_val_t)
            val_loss_T = criterion(val_pred_T, y_T_val_t)
            val_loss_T1 = criterion(val_pred_T1, y_T1_val_t)
            val_loss = (val_loss_T + val_loss_T1).item()
        
        history['train_loss'].append(epoch_loss)
        history['val_loss'].append(val_loss)
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{epochs}] - Train Loss: {epoch_loss:.6f}, Val Loss: {val_loss:.6f}")
    
    print("\n✓ SST训练完成")
    return history


# 训练模型
sst_history = train_sst(
    sst_model,
    X_train, y_T_train, y_T1_train,
    X_val, y_T_val, y_T1_val,
    epochs=50,
    batch_size=32,
    lr=0.001
)

# 绘制训练曲线
plt.figure(figsize=(10, 4))
plt.plot(sst_history['train_loss'], label='Train Loss')
plt.plot(sst_history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('SST Training History')
plt.show()

## 🔍 Stage 2: 特征提取

从训练好的SST模型中提取内部特征

In [ ]:
# 此cell继续完整实现...
# 由于篇幅限制，我会在后续cell中继续补充

print("⏭️ 继续下一步...")